<a href="https://colab.research.google.com/github/c-wegman/ABW/blob/main/ABW_2022_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell first!
class _CgAnswers: ...
answers = _CgAnswers()


# Analytics for a better world
## Assignment 6: Linear programming (with indexed variables)

* This assignment is made for auto-grading with CodeGrade in Canvas
* Write your code
* **NEVER TOUCH THE CODE WHERE IT IS MARKED "DO NOT CHANGE"**
* Submit through Canvas: [Link](https://canvas.uva.nl/courses/32521/assignments/362161)
* Check the feedback

In [9]:
# DO NOT CHANGE
import sys
at_colab = "google.colab" in sys.modules

if at_colab:
    import shutil
    if not shutil.which("pyomo"):
        !pip install -q pyomo
        assert(shutil.which("pyomo"))
    if not shutil.which('/usr/bin/cbc'):
        !apt-get install -y -qq coinor-cbc
        assert(shutil.which('/usr/bin/cbc'))

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

# Question 1 - Larry Edison (5 points)

Consider the exercise 3.4-8 of the book by Hillier and Lieberman:

Larry Edison is the director of the Computer Center for Buckly College. He now needs to schedule the staffing of the center. It is open from 8 AM until midnight. Larry has monitored the usage of the center at various times of the day, and determined that the following number of computer consultants are required:

| Shift |Time of Day | Minimum Number of Consultants |
|-------|------------|-------------------------------|
|   A   |8 AM - noon | 4 |
|   B   |Noon - 4 PM | 8 |
|   C   |4 PM - 8 PM | 10 |
|   D   |8 PM - midnight | 6 |

Two types of computer consultants can be hired: full-time and part-time. The full-time consultants work for 8 consecutive hours in any of the following shifts: morning (8 AM - 4 PM), afternoon (noon - 8 PM), and evening  (4 PM - midnight). Full-time consultants are paid €40 per hour. Part-time consultants can be hired to work any of the four shifts listed in the table above. Part-time consultants are paid €30 per hour. An additional requirement is that during every time period, there must be at least two full-time consultants on duty for every part-time consultant on duty.

Larry would like to determine how many full-time and how many part-time workers should work each shift to meet the above requirements at the minimum possible cost. Formulate a linear programming model with indexed variables, and solve it using Pyomo. Make sure that the solution can only take integer values, since it is not possible to hire a worker for only part of a shift.

Write a function:
* named `model_larry_edison`, with no arguments
* returning a pyomo model
* The model should have:
    * one objective, named `wage`, minimizing the amount of wage that has to be paid.
    * two variables: 
        * `full_time` with three indices, one for each full time shift, indicating how many full-time workers are on duty during that shift.
        * `part_time` with four indices, one for each part time shift, indicating how many part-time workers are on duty during that shift.
    * eight constraints
        * four (`A_total`, `B_total`, `C_total`, `D_total`) making sure the number of required consultants is satisfied for each 4-hour shift.
        * four (`A_ratio`, `B_ratio`, `C_ratio`, `D_ratio`) making sure that at each 4-hour shift, the number of full time consultants is at least twice the number of part time consultants.
    
**POINTS** 
* Correctly defining the objective function: 1 point
* Correctly defining the constraints: 1 point
* Getting the expected solution when solving the model: 3 points

In [3]:
# DO NOT CHANGE
import pyomo.environ as pyo

In [4]:
### BEGIN SOLUTION
def model_larry_edison():

  model           = pyo.ConcreteModel('LarryEdison')
  model.fulltime  = pyo.Set(initialize=['morning', 'afternoon', 'evening'])
  model.parttime  = pyo.Set(initialize=['A', 'B', 'C', 'D'])
  model.full_time = pyo.Var(model.fulltime, within=pyo.NonNegativeIntegers )
  model.part_time = pyo.Var(model.parttime, within=pyo.NonNegativeIntegers )

  model.wage      = pyo.Objective(sense= pyo.minimize, expr = pyo.quicksum(40*8*model.full_time[x] for x in model.fulltime) + pyo.quicksum(30*4*model.part_time[x] for x in model.parttime))

  model.A_total   = pyo.Constraint(expr = model.full_time['morning']+model.part_time['A'] >= 4)
  model.B_total   = pyo.Constraint(expr = model.full_time['morning']+model.full_time['afternoon']+model.part_time['B'] >= 8)
  model.C_total   = pyo.Constraint(expr = model.full_time['afternoon']+model.full_time['evening']+model.part_time['C'] >= 10)
  model.D_total   = pyo.Constraint(expr = model.full_time['evening']+model.part_time['D'] >= 6)

  model.A_ratio   = pyo.Constraint(expr = model.full_time['morning'] >= 2*model.part_time['A'])
  model.B_ratio   = pyo.Constraint(expr = model.full_time['morning']+model.full_time['afternoon'] >= 2*model.part_time['B'])
  model.C_ratio   = pyo.Constraint(expr = model.full_time['afternoon']+model.full_time['evening'] >= 2*model.part_time['C'])
  model.D_ratio   = pyo.Constraint(expr = model.full_time['evening'] >= 2*model.part_time['D'])

  return model

model_larry_edison()
### END SOLUTION

In [5]:
# DO NOT CHANGE 
answers.Q1_obj = model_larry_edison
answers.Q1_con = model_larry_edison
answers.Q1_out = model_larry_edison

# Question 2 - Oxbridge (5 points)

Consider the exercise 3.4-14 of the book by Hillier and Lieberman:

Oxbridge University maintains a powerful mainframe computer for research use by its faculty, Ph.D. students, and research associates. During all working hours, an operator must be available to operate and maintain the computer, as well as to perform some programming services. Beryl Ingram, the director of the computer facility, oversees the operation. It is now the beginning of the fall semester, and Beryl is confronted with the problem of assigning different working hours to her operators. Because all the operators are currently enrolled in the university, they are available to work only a limited number of hours each day. The table below shows, for each operator, both the wage per hour and the maximum number of working hours that the operator is available per week.

| Operators | Wage Rate | Mon. | Tue. | Wed. | Thurs. | Fri. |
|-----------|-----------|------|------|------|--------|------|
| K.C.     | €25 / hour  |  6   |  0   |  6   |  0     |  6   |
| D.H.     | €26 / hour  |  0   |  6   |  0   |  6     |  0   |
| H.B.     | €24 / hour  |  4   |  8   |  4   |  0     |  4   |
| S.C.     | €23 / hour  |  5   |  5   |  5   |  0     |  5   |
| K.S.     | €28 / hour  |  3   |  0   |  3   |  8     |  0   |
| N.K.     | €30 / hour  |  0   |  0   |  0   |  6     |  2   |


There are six operators (four undergraduate students and two graduate students). They all have different wage rates because of differences in their experience with computers and in their programming ability. Each operator is guaranteed a certain minimum number of hours per week that will maintain an adequate knowledge of the operation. This level is set at 8 hours per week for the undergraduate students (K.C., D.H., H.B., and S.C.) and 7 hours per week for the graduate students (K.S. and N.K.). The computer facility is to be open for operation from 8 AM to 10 PM Monday through Friday with exactly one operator on duty during each of these hours. Because of a tight budget, Beryl has to minimize cost. She wishes to determine the number of hours she should assign to each operator on each day. Formulate a linear programming model for this problem.

Write a function:
* named `model_oxbridge`, with no arguments
* returning a pyomo model
* The model should have:
    * one indexed variable named `x`, stating the number of hours per day that each operator should be on duty.
        * Make sure that this variable is indexed first by the operator names (K.C., D.H., etc.), and then by the days (Mon, Tue, etc.). These names should be equal to the index- and column names in the `data` table as defined in the next cell.
    * two indexed constraints
        * `min_hours`: make sure each operator is working their minimal number of hours.
        * `day_total`: make sure the total number of hours worked each day is exactly 14 hours (8 AM to 10 PM).
    * one objective, named `wage`.

**POINTS** 
* Correctly defining the objective function: 1 point
* Correctly defining the constraints: 2 points
* Getting the expected solution when solving the model: 2 points

**POINTS** 
* Correctly defining the objective function: 1 point
* Correctly defining the constraints: 2 points
* Getting the expected solution when solving the model: 2 points

In [6]:
# DO NOT CHANGE
import pyomo.environ as pyo
import pandas as pd
from io import StringIO

data = StringIO('''Operator;Wage;Mon;Tue;Wed;Thu;Fri;MinHours
K.C.; 25 ; 6 ; 0 ; 6 ; 0 ; 6 ; 8
D.H.; 26 ; 0 ; 6 ; 0 ; 6 ; 0 ; 8
H.B.; 24 ; 4 ; 8 ; 4 ; 0 ; 4 ; 8
S.C.; 23 ; 5 ; 5 ; 5 ; 0 ; 5 ; 8
K.S.; 28 ; 3 ; 0 ; 3 ; 8 ; 0 ; 7
N.K.; 30 ; 0 ; 0 ; 0 ; 6 ; 2 ; 7 ''')

df = pd.read_csv(data, sep=";", index_col='Operator')
days = ['Mon','Tue','Wed','Thu','Fri']
df[days] = df[days].astype(float)

In [10]:
from pyomo.core.base.set_types import NonNegativeReals
### BEGIN SOLUTION
def model_oxbridge():

  model           = pyo.ConcreteModel('Oxbridge')
  model.operator  = pyo.Set(initialize=df.index)
  model.days      = pyo.Set(initialize=days)

  def availability_bound(model, d, o):
    return 0,df[o][d]

  model.x         = pyo.Var(model.operator, model.days, within=NonNegativeReals, bounds=availability_bound)

  @model.Constraint( model.operator )
  def min_hours( model, o ):
    return pyo.quicksum( model.x[o,d] for d in model.days ) >= df.MinHours[o]
  
  @model.Constraint( model.days )
  def day_total( model, d ):
    return pyo.quicksum( model.x[o,d] for o in model.operator ) == 14

  #@model.Constraint( model.days, model.operator )
  #def availability( model, d, o ):
  #  return model.x[o,d] <= df[d][o]

  
  
  model.wage      = pyo.Objective(sense = pyo.minimize, expr = pyo.quicksum(df.Wage[o]*model.x[o,d] for o in model.operator for d in model.days))

  return model

#model_oxbridge().pprint()
model = model_oxbridge()
results = pyo.SolverFactory('cbc',executable='/usr/bin/cbc').solve(model)
print(results.solver.status, results.solver.termination_condition )
print( model.wage(), model.x.extract_values() )
### END SOLUTION

ok optimal
1755.0 {('K.C.', 'Mon'): 2.0, ('K.C.', 'Tue'): 0.0, ('K.C.', 'Wed'): 4.0, ('K.C.', 'Thu'): 0.0, ('K.C.', 'Fri'): 3.0, ('D.H.', 'Mon'): 0.0, ('D.H.', 'Tue'): 2.0, ('D.H.', 'Wed'): 0.0, ('D.H.', 'Thu'): 6.0, ('D.H.', 'Fri'): 0.0, ('H.B.', 'Mon'): 4.0, ('H.B.', 'Tue'): 7.0, ('H.B.', 'Wed'): 4.0, ('H.B.', 'Thu'): 0.0, ('H.B.', 'Fri'): 4.0, ('S.C.', 'Mon'): 5.0, ('S.C.', 'Tue'): 5.0, ('S.C.', 'Wed'): 5.0, ('S.C.', 'Thu'): 0.0, ('S.C.', 'Fri'): 5.0, ('K.S.', 'Mon'): 3.0, ('K.S.', 'Tue'): 0.0, ('K.S.', 'Wed'): 1.0, ('K.S.', 'Thu'): 3.0, ('K.S.', 'Fri'): 0.0, ('N.K.', 'Mon'): 0.0, ('N.K.', 'Tue'): 0.0, ('N.K.', 'Wed'): 0.0, ('N.K.', 'Thu'): 5.0, ('N.K.', 'Fri'): 2.0}


In [8]:
# DO NOT CHANGE 
answers.Q2_obj = model_oxbridge
answers.Q2_con = model_oxbridge
answers.Q2_out = model_oxbridge